<a href="https://colab.research.google.com/github/angegiulio/IFC-Cost-Calculation/blob/main/NB_Kostensch%C3%A4tzung_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install ifcopenshell pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.3 MB/s eta 0:00:00


In [ ]:

ifc_file = ifcopenshell.open(r"NB_AR_SB_M ARCHI-RE.ifc")  # Replace with actual file path


✅ Exported wall data to wall_surface_summary.csv


In [21]:
import ifcopenshell
import pandas as pd
import re

# === Load IFC file ===
ifc_file = ifcopenshell.open(r"NB_AR_SB_M ARCHI-RE.ifc")

# === Helper functions ===

def get_clean_name(name):
    """Extract the part of the name between the first and second colon."""
    if not name:
        return None
    parts = name.split(":")
    return parts[1] if len(parts) >= 3 else name


def get_building_of_element(element):
    """Get the building to which the element belongs."""
    for rel in element.ContainedInStructure or []:
        for obj in rel.RelatingStructure.RepresentationContexts or []:
            if obj.is_a("IfcBuilding"):
                return obj.Name
        if rel.RelatingStructure.is_a("IfcBuilding"):
            return rel.RelatingStructure.Name
        elif rel.RelatingStructure.is_a("IfcBuildingStorey"):
            # Go up one level to building
            parent = rel.RelatingStructure.Decomposes
            while parent:
                parent_obj = parent[0].RelatingObject if parent else None
                if parent_obj and parent_obj.is_a("IfcBuilding"):
                    return parent_obj.Name
                parent = parent_obj.Decomposes if parent_obj else None
    return "Unknown"

def get_building_of_element(element):
    """Traverse up containment to find the IfcBuilding."""
    if not element.ContainedInStructure:
        return "Unknown"

    for rel in element.ContainedInStructure:
        structure = rel.RelatingStructure
        while structure:
            if structure.is_a("IfcBuilding"):
                return structure.Name
            if hasattr(structure, "Decomposes") and structure.Decomposes:
                structure = structure.Decomposes[0].RelatingObject
            else:
                break
    return "Unknown"


def get_surface_area(wall):
    """Get wall area from 'Dimensions' PSet > 'Area'."""
    value = get_property_value(wall, "Dimensions", "Area")
    if value:
        try:
            return float(value)
        except:
            return 0.0
    return 0.0


# === Geometry settings ===
import ifcopenshell.geom
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)

# === Extract walls ===
walls = ifc_file.by_type("IfcWall")

data = []

for wall in walls:
    clean_name = get_clean_name(wall.Name)
    building_name = get_building_of_element(wall)

    fire_resistance = get_property_value(wall, "Fire Protection", "Feuerwiderstand Soll")
    if not fire_resistance:
        fire_resistance = "0"

    surface = get_surface_area(wall)

    data.append({
        "Building": building_name,
        "WallName": clean_name,
        "FireResistance": str(fire_resistance),  # Make sure it's string for grouping
        "Surface_m2": surface
    })


# === Convert to DataFrame ===
df = pd.DataFrame(data)

# === Group and summarize ===
grouped = df.groupby(["WallName", "FireResistance"]).agg({
    "Surface_m2": "sum"
}).reset_index()

print("\n=== Unique combinations of Wall Name and Fire Resistance with Surface Areas (m²) ===")
print(grouped)

# === Optional: show grouped by building ===
print("\n=== Wall Names grouped by Building ===")
wall_names_by_building = df.groupby("Building")["WallName"].unique()
print(wall_names_by_building)



=== Unique combinations of Wall Name and Fire Resistance with Surface Areas (m²) ===
                    WallName FireResistance  Surface_m2
0        ARC_FAC_PCC_FBT_STZ              0  334.441592
1        ARC_ROF_CON_ATK_DBT              0   94.609875
2         ARC_ROF_CON_ATK_VB              0  184.446158
3        ARC_WAL_MSN_BRK_EGN              0    1.619015
4    ARC_WAN_CON_CPC_OBL_100              0   87.117344
..                       ...            ...         ...
222                  Walls 1              0   28.003173
223                 Walls 16              0    0.173039
224                 Walls 20              0    0.178630
225                  Walls 7              0  103.550798
226         Wand Dachlicht-1              0  164.521834

[227 rows x 3 columns]

=== Wall Names grouped by Building ===
Building
Sanierungsbau    [WAT_STB_ORT_140, WAT_STB_ORT_120, WAT_STB_ORT...
Name: WallName, dtype: object


In [22]:
grouped.to_csv("wall_fire_resistance_summary.csv", index=False)